<a href="https://colab.research.google.com/github/subir2210/PROJECTS/blob/main/Sample%20Project/Phishing-Website-Detection-by-Machine-Learning-Techniques-master/lab/URL%20Feature%20Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phishing Website Detection Feature Extraction**

*Final project of AI & Cybersecurity Course*

# **1. Objective:**
A phishing website is a common social engineering method that mimics trustful uniform resource locators (URLs) and webpages. The objective of this notebook is to collect data & extract the selctive features form the URLs.

*This project is worked on Google Collaboratory.*



# **2. Collecting the Data:**
For this project, we need a bunch of urls of type legitimate (0) and phishing (1). 

The collection of phishing urls is rather easy because of the opensource service called PhishTank. This service provide a set of phishing URLs in multiple formats like csv, json etc. that gets updated hourly. To download the data: https://www.phishtank.com/developer_info.php

For the legitimate URLs, I found a source that has a collection of benign, spam, phishing, malware & defacement URLs. The source of the dataset is University of New Brunswick, https://www.unb.ca/cic/datasets/url-2016.html. The number of legitimate URLs in this collection are 35,300. The URL collection is downloaded & from that, *'Benign_list_big_final.csv'* is the file of our interest. This file is then uploaded to the Colab for the feature extraction. 


## **2.1. Phishing URLs:**

The phishing URLs are collected from the PhishTank from the link provided. The csv file of phishing URLs is obtained by using wget command. After downlaoding the dataset, it is loaded into a DataFrame.

In [2]:
#importing required packages for this module
import pandas as pd

In [4]:
#Downloading the phishing URLs file
!wget http://data.phishtank.com/data/online-valid.csv

--2021-04-14 12:22:23--  http://data.phishtank.com/data/online-valid.csv
Resolving data.phishtank.com (data.phishtank.com)... 104.17.177.85, 104.16.101.75, 2606:4700::6810:654b, ...
Connecting to data.phishtank.com (data.phishtank.com)|104.17.177.85|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.phishtank.com/data/online-valid.csv [following]
--2021-04-14 12:22:23--  https://data.phishtank.com/data/online-valid.csv
Connecting to data.phishtank.com (data.phishtank.com)|104.17.177.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1750zhbc38ec0.cloudfront.net/datadumps/verified_online.csv?Expires=1618402953&Signature=FgM5hts2jkyCAE3hK5Z0dLXnVhLEkIl7rSf9aYOXirfk-ZXIhPzpDdoz2KbFv5fsxnhxN4ZJaI00C-ukrXFBqV8RZzOfBJrNDK06Lkeyd~Co11uRqDojg~vFNZMreOkTfSbDtZfLqFoj1kabojJGYeV6LoR5t2KhpjvMCXZMOjEdUUAXI6s8tGyau9JrPFoZCnVxxJvC6GaPI0z82PVBcQSbKmx7K39zbjDOOlExI14Gy7ep4-kbsQvtVHgBire9FCopTlA8QbqKWBchps-3e342m

The above command downlaods the file of phishing URLs, *online-valid.csv* and stores in the */content/* folder. 

In [5]:
#loading the phishing URLs data to dataframe
data0 = pd.read_csv("online-valid.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7069633,http://home-web-id.com,http://www.phishtank.com/phish_detail.php?phis...,2021-04-14T11:10:19+00:00,yes,2021-04-14T11:15:54+00:00,yes,Other
1,7069632,http://home-web-id.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-04-14T11:10:14+00:00,yes,2021-04-14T11:15:54+00:00,yes,Other
2,7069619,https://olx.pl-paydelivery.xyz,http://www.phishtank.com/phish_detail.php?phis...,2021-04-14T10:43:46+00:00,yes,2021-04-14T10:48:24+00:00,yes,Other
3,7069618,https://olx.pl-paydelivery.cyou,http://www.phishtank.com/phish_detail.php?phis...,2021-04-14T10:43:36+00:00,yes,2021-04-14T10:48:24+00:00,yes,Other
4,7069617,https://olx.pl-paydelivery.work,http://www.phishtank.com/phish_detail.php?phis...,2021-04-14T10:43:35+00:00,yes,2021-04-14T10:47:26+00:00,yes,Other


In [6]:
data0.shape

(10057, 8)

So, the data has thousands of phishing URLs. But the problem here is, this data gets updated hourly. Without getting into the risk of data imbalance, I am considering a margin value of 10,000 phishing URLs & 5000 legitimate URLs. 

Thereby, picking up 5000 samples from the above dataframe randomly.

In [7]:
#Collecting 5,000 Phishing URLs randomly
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6837481,https://linktr.ee/TencentGamesss,http://www.phishtank.com/phish_detail.php?phis...,2020-11-06T10:10:48+00:00,yes,2020-11-06T13:16:02+00:00,yes,Other
1,6954912,https://beansbulletsbandagesandyou.com/bullets...,http://www.phishtank.com/phish_detail.php?phis...,2021-02-06T13:36:20+00:00,yes,2021-02-06T13:38:53+00:00,yes,Allegro
2,7058642,http://52.235.24.152/Finance/bmo-mobile,http://www.phishtank.com/phish_detail.php?phis...,2021-04-05T15:00:26+00:00,yes,2021-04-05T16:30:13+00:00,yes,Other
3,6863227,https://tinyurl.com/y2czr3ag,http://www.phishtank.com/phish_detail.php?phis...,2020-11-25T19:32:17+00:00,yes,2020-11-26T02:34:08+00:00,yes,Other
4,4944323,http://volarevic.com/sass/cab_jo/cab/cab/cab_jo/,http://www.phishtank.com/phish_detail.php?phis...,2017-04-12T06:43:08+00:00,yes,2017-07-13T03:51:56+00:00,yes,Other


In [8]:
phishurl.shape

(5000, 8)

In [10]:
phishurl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   phish_id           5000 non-null   int64 
 1   url                5000 non-null   object
 2   phish_detail_url   5000 non-null   object
 3   submission_time    5000 non-null   object
 4   verified           5000 non-null   object
 5   verification_time  5000 non-null   object
 6   online             5000 non-null   object
 7   target             5000 non-null   object
dtypes: int64(1), object(7)
memory usage: 312.6+ KB


In [11]:
phishurl.nunique()

phish_id             5000
url                  5000
phish_detail_url     5000
submission_time      4851
verified                1
verification_time    4155
online                  1
target                 72
dtype: int64

As of now we collected 5000 phishing URLs. Now, we need to collect the legitimate URLs.

## **2.2. Legitimate URLs:**

From the uploaded *Benign_list_big_final.csv* file, the URLs are loaded into a dataframe.

In [17]:
#Loading legitimate files 
data1 = pd.read_csv("https://raw.githubusercontent.com/subir2210/PROJECTS/main/Sample%20Project/Phishing-Website-Detection-by-Machine-Learning-Techniques-master/DataFiles/1.Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()


,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


As stated above, 5000 legitimate URLs are randomaly picked from the above dataframe.

# **3. Feature Extraction:**

In this step, features are extracted from the URLs dataset.

The extracted features are categorized into


1.   Address Bar based Features
2.   Domain based Features
3.   HTML & Javascript based Features



### **3.1. Address Bar Based Features:**

Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.


*   Domain of URL
*   IP Address in URL
*   "@" Symbol in URL
*   Length of URL
*   Depth of URL
*   Redirection "//" in URL
*   "http/https" in Domain name
*   Using URL Shortening Services “TinyURL”
*   Prefix or Suffix "-" in Domain

Each of these features are explained and the coded below:

#### **3.1.1. Domain of the URL**
Here, we are just extracting the domain present in the URL. This feature doesn't have much significance in the training. May even be dropped while training the model.

#### **3.1.2. IP Address in the URL**

Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).



#### **3.1.3. "@" Symbol in URL**

Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol. 

If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

#### **3.1.4. Length of URL**

Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.

If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

#### **3.1.5. Depth of URL**

Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

#### **3.1.6. Redirection "//" in URL**

Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else 0 (legitimate).

#### **3.1.7. "http/https" in Domain name**

Checks for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.

If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

#### **3.1.8. Using URL Shortening Services “TinyURL”**

URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL. 

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [34]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

#### **3.1.9. Prefix or Suffix "-" in Domain**

Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage. 

If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

### **3.2. Domain Based Features:**

Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

*   DNS Record
*   Website Traffic 
*   Age of Domain
*   End Period of Domain

Each of these features are explained and the coded below:

#### **3.2.1. DNS Record**

For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname. 
If the DNS record is empty or not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

#### **3.2.2. Web Traffic**

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

#### **3.2.3. Age of Domain**

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

#### **3.2.4. End Period of Domain**

This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less  for this project. 

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

## **3.3. HTML and JavaScript based Features**

Many features can be extracted that come under this category. Out of them, below mentioned were considered for this project.

*   IFrame Redirection
*   Status Bar Customization
*   Disabling Right Click
*   Website Forwarding

Each of these features are explained and the coded below:

### **3.3.1. IFrame Redirection**

IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation. 

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

### **3.3.2. Status Bar Customization**

Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the “onMouseOver” event, and check if it makes any changes on the status bar

If the response is empty or onmouseover is found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

### **3.3.3. Disabling Right Click**

Phishers use JavaScript to disable the right-click function, so that users cannot view and save the webpage source code. This feature is treated exactly as “Using onMouseOver to hide the Link”. Nonetheless, for this feature, we will search for event “event.button==2” in the webpage source code and check if the right click is disabled.

If the response is empty or onmouseover is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).




### **3.3.4. Website Forwarding**
The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times. 




## **4. Computing URL Features**

Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

### **4.1. Legitimate URLs:**

Now, feature extraction is done on legitimate URLs.

### **4.2. Phishing URLs:**

Now, feature extraction is performed on phishing URLs.

## **5. Final Dataset**

In the above section we formed two dataframes of legitimate & phishing URL features. Now, we will combine them to a single dataframe and export the data to csv file for the Machine Learning training done in other notebook. 

## **6. Conclusion**

With this the objective of this notebook is achieved. We finally extracted 18 features for 10,000 URL which has 5000 phishing & 5000 legitimate URLs.

## **7. References**

* https://archive.ics.uci.edu/ml/datasets/Phishing+Websites 

# New Section